In [1]:
import sys
import os
# Append the library path to PYTHONPATH, so library can be imported.
sys.path.append(os.path.dirname(os.getcwd()))
import shutil
import datetime

import numpy as np
import pandas as pd

from library import stoxx as st

In [2]:
%run setup.py

%load_ext autoreload
%autoreload 2


np.random.seed(RANDOM_SEED)

In [3]:
%run Load_Clean_aux.py

C:\Users\Weiguan\Anaconda3\envs\tfenv\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


We remove samples within one hedging offset before OPTION expiry. 196365 samples ( 5.17%) are removed. We have  94.83% of original data left, which is 3600717.
We remove samples when the next trade is not available. 1699921 samples ( 47.21%) are removed. We have  50.06% of original data left, which is 1900796.
We remove samples when the matching tol is larger than 0.1H. 1037821 samples ( 54.60%) are removed. We have  22.73% of original data left, which is 862975.
We remove NA implvol0. 0 samples ( 0.00%) are removed. We have  22.73% of original data left, which is 862975.
We remove unreasonable implvol0. 4 samples ( 0.00%) are removed. We have  22.73% of original data left, which is 862971.
We remove the first and the last part of day. 217719 samples ( 25.23%) are removed. We have  16.99% of original data left, which is 645252.
We remove samples within one hedging offset before FUTURES expiry. 14094 samples ( 2.18%) are removed. We have  16.62% of original data left, which is 631158.
W

In [4]:
raw_dir = DATA_DIR + 'RawData/'

In [5]:
# Liquidity file
df_liq = pd.read_csv(
    raw_dir + 'liquidity/EurexRTAIOC.csv', 
    delimiter=';',
    parse_dates={'DateTime_LIQ': ['date', 'MDStatisticTime']})

- We do not need to use refData.csv, since they have already been merged perviously.

In [6]:
df_liq.head()

,DateTime_LIQ,SecurityID,MDStatisticID,MDStatisticName,MDStatisticValue
0,2016-04-25 08:44:56.579274344,26577,NaN,LAST_TRADE_PRICE,2206000000000000e-13
1,2016-04-25 08:44:56.579274344,26577,NaN,LAST_TRADE_QUANTITY,1000000000000000e-13
2,2016-04-25 08:44:56.579274344,26577,NaN,TRADE_MATCH_IDENTIFIER,1262.000000
3,2016-04-25 08:44:56.579274344,26577,NaN,SIDE,B
4,2016-04-25 08:44:56.579274344,26577,480.0,LIQUIDITY_IND,5.000000000000000


Some rows have same DateTime, SecurityID and MDStatisticName, but different ‘MDStatisticValue’.  In principle, one should take the average of the ‘MDStatisticValue’ when there are duplicates. But the difficulty is in the `side` , which is categorical.  Imagine all of the ` DateTime, SecurityID and MDStatisticName,` are same, but side are different, you don’t know which rows correspond to a given `side`. Maybe One can use the the locations  (e.g. take the previous 4 rows for this side). Leave this for the future, and I don’t think this would matter much, since the duplicates only count for 0.3%. **Come back to this issue if the general results is promising.** 



In [7]:
# Drop all rows where these three entries are duplicated.
df_liq = df_liq.loc[~df_liq.duplicated(['DateTime_LIQ', 'MDStatisticName', 'SecurityID'], keep=False)]

In [8]:
# Liquidity data is arranged in a stacked table. So we need to unstack.
df_liq = df_liq.set_index(['DateTime_LIQ', 'SecurityID', 'MDStatisticName'])

df_liq = df_liq.unstack()['MDStatisticValue']

In [9]:
cols = ['LAST_TRADE_PRICE', 'LAST_TRADE_QUANTITY', 'LIQUIDITY_IND']
df_liq[cols] = df_liq[cols].apply(pd.to_numeric)

cols = ['TRADE_MATCH_IDENTIFIER']
df_liq[cols] = df_liq[cols].astype('category')

# After unstacking, the table is like the following
df_liq.head()

,MDStatisticName,LAST_TRADE_PRICE,LAST_TRADE_QUANTITY,LIQUIDITY_IND,SIDE,TRADE_MATCH_IDENTIFIER
DateTime_LIQ,SecurityID,,,,,
2016-04-25 07:00:56.526222102,1678410,1.7,20.0,80.0,S,4.000000
2016-04-25 07:01:28.148100232,1690717,11.1,1.0,19.0,B,6.000000
2016-04-25 07:03:15.504480581,496215,81.0,1.0,100.0,B,8.000000
2016-04-25 07:03:34.793352477,1550511,11.3,3.0,97.0,B,9.000000
2016-04-25 07:03:46.157874873,1678446,53.0,7.0,93.0,B,10.000000


- 'MDStatisticID' was dropped by previous step. It only has two values, NaN or 480. Whenever 'MDStatisticID' is only 480 for rows that correspond to `LIQUIDITY_IND`.

In [10]:
df_liq = df_liq.reset_index()

The `Side` indicator should means the same thing as the aggressorside in the option file. The `AgressorSide` uses 1.0 for buy and 2.0 for sell. So i change the side in liquidatity file to same values, so that the keys can be matched later.

In [11]:
df_liq.loc[df_liq['SIDE'] == 'B', 'SIDE'] = '1.0'
df_liq.loc[df_liq['SIDE'] == 'S', 'SIDE'] = '2.0'

#### Step 5.3: Append liquidity indicators
Here, we append available liquidity infos to options that are common in both files. 

Liquidity file and option file do not have the same starting time. So we will merge the liquidity indicators into the option file, when it is possible to do so. That means, when an option is in both files. This process is subject to a maximum time lag. If liquidity indicators are earlier than option trades by a lag, they are not accepted.

For each common group, we now need to find the right place to append indicators for each trade of options. We take the info prior to the order actually came.

In [12]:
new_cols = ['LAST_TRADE_PRICE', 'LAST_TRADE_QUANTITY', 
            'LIQUIDITY_IND', 'TRADE_MATCH_IDENTIFIER', 'DateTime_LIQ']

In [13]:
df = df.reindex(columns=df.columns.tolist() + new_cols)

liq_groupped = df_liq.groupby(['SecurityID', 'SIDE'])
liq_ids = liq_groupped.groups.keys()
df_groupped = df.groupby(['SecurityID', 'AggressorSide'])
df_ids = df_groupped.groups.keys()

common_groups = list(set(liq_ids).intersection(set(df_ids)))

In [14]:
for ele in common_groups:
    liq_tmp = liq_groupped.get_group(ele).copy() # get dataframe
    df_tmp = df_groupped.get_group(ele).copy()
    id_lable = df_groupped.groups[ele].copy() # get idx labels
    
    liq_tmp.sort_values(by='DateTime_LIQ', inplace=True)
    liq_tmp.reset_index(drop=True, inplace=True)
    
    ts = df_tmp['ExecuteTime0'] 
    
    id_int = liq_tmp['DateTime_LIQ'].searchsorted(df_tmp['ExecuteTime0'])
    id_int = np.maximum(id_int - 1, 0)
    print(ele)
    
    # if the indicators are away for more than half an hour range, we discard
    df.loc[id_lable, new_cols] = liq_tmp.loc[id_int, new_cols].values
    na_index1 = id_lable[liq_tmp.loc[id_int, 'DateTime_LIQ'].values  < (ts - MAX_LAG_LIQ).values]
    na_index2 = id_lable[liq_tmp.loc[id_int, 'DateTime_LIQ'].values > ts.values]
    df.loc[na_index1.union(na_index2), new_cols] = np.nan

(2228225, '2.0')
(2850817, '2.0')
(27104, '2.0')
(2271712, '2.0')
(1992602, '1.0')
(2727560, '1.0')
(2757894, '2.0')
(3062831, '1.0')
(1733564, '1.0')
(2241468, '1.0')
(2028768, '1.0')
(1844940, '1.0')
(2284136, '1.0')
(3239285, '1.0')
(2078054, '1.0')
(2413926, '1.0')
(2850818, '1.0')
(1294254, '1.0')
(1786265, '1.0')
(2789739, '2.0')
(2432754, '2.0')
(2320150, '2.0')
(2466963, '2.0')
(1893978, '1.0')
(2727550, '2.0')
(2078063, '2.0')
(2413935, '2.0')
(2432751, '1.0')
(2932463, '1.0')
(27101, '1.0')
(2271709, '1.0')
(2659461, '2.0')
(2133354, '1.0')
(1678455, '1.0')
(2121099, '2.0')
(3134212, '1.0')
(27137, '1.0')
(3134194, '1.0')
(2284135, '2.0')
(2478235, '1.0')
(1844937, '2.0')
(1733544, '2.0')
(1117373, '1.0')
(467033, '1.0')
(467018, '1.0')
(2605130, '1.0')
(2932452, '1.0')
(2432740, '1.0')
(2271698, '1.0')
(27090, '1.0')
(2659457, '2.0')
(1948795, '1.0')
(1678444, '1.0')
(1974880, '2.0')
(3062892, '1.0')
(1786389, '1.0')
(27142, '1.0')
(2028785, '1.0')
(1786246, '2.0')
(2478255,

(2727555, '2.0')
(858044, '1.0')
(1786384, '2.0')
(1786437, '2.0')
(1978533, '1.0')
(2271698, '2.0')
(2757971, '1.0')
(3134221, '2.0')
(2028797, '2.0')
(2757885, '2.0')
(2228208, '1.0')
(1733550, '1.0')
(1948823, '1.0')
(2284153, '2.0')
(2390649, '2.0')
(26675, '1.0')
(3134194, '2.0')
(3253825, '2.0')
(1844926, '1.0')
(1914183, '2.0')
(2757959, '2.0')
(1550505, '1.0')
(2659413, '2.0')
(2078024, '1.0')
(1596386, '2.0')
(2090819, '1.0')
(2554223, '2.0')
(1596386, '1.0')
(2727566, '1.0')
(2554233, '1.0')
(2134628, '2.0')
(2554265, '2.0')
(2757965, '2.0')
(1060487, '1.0')
(2878210, '2.0')
(2090813, '2.0')
(1844930, '1.0')
(2271695, '1.0')
(2757905, '1.0')
(1294285, '1.0')
(2478203, '1.0')
(1893984, '2.0')
(1294270, '2.0')
(3062900, '2.0')
(2932457, '2.0')
(2659472, '1.0')
(858033, '2.0')
(2074958, '1.0')
(2466960, '2.0')
(2605116, '1.0')
(2932438, '1.0')
(2023617, '1.0')
(1678430, '1.0')
(1992615, '2.0')
(2809333, '1.0')
(2554260, '1.0')
(2554237, '2.0')
(1418109, '1.0')
(2165674, '2.0')
(

(2605121, '1.0')
(26674, '1.0')
(2157792, '1.0')
(2932443, '1.0')
(2271689, '1.0')
(1678435, '1.0')
(2659407, '2.0')
(3062883, '1.0')
(2074961, '2.0')
(858030, '1.0')
(1435115, '1.0')
(1893936, '2.0')
(1786250, '1.0')
(2478231, '1.0')
(1786424, '1.0')
(2554219, '2.0')
(2320114, '2.0')
(1418111, '2.0')
(1570316, '2.0')
(490924, '1.0')
(1894014, '2.0')
(1992615, '1.0')
(467014, '1.0')
(2228194, '2.0')
(2605126, '1.0')
(467014, '2.0')
(2028739, '2.0')
(2432764, '2.0')
(2605126, '2.0')
(2228194, '1.0')
(1948805, '2.0')
(2121113, '2.0')
(2878194, '1.0')
(2554270, '1.0')
(467008, '2.0')
(2284149, '2.0')
(2390645, '2.0')
(1418107, '2.0')
(2478220, '1.0')
(2605120, '2.0')
(1690726, '2.0')
(683471, '1.0')
(1992619, '1.0')
(1550501, '2.0')
(858027, '2.0')
(2415864, '1.0')
(1844921, '2.0')
(2228214, '1.0')
(1418136, '2.0')
(1733581, '1.0')
(1948801, '2.0')
(2121086, '1.0')
(2659436, '1.0')
(2821993, '1.0')
(3000346, '1.0')
(1733566, '2.0')
(2478196, '2.0')
(1786374, '1.0')
(2028803, '1.0')
(27578

(2252969, '2.0')
(2121096, '2.0')
(1733602, '1.0')
(2878191, '1.0')
(881404, '2.0')
(2694347, '1.0')
(1690704, '2.0')
(2478213, '2.0')
(1786265, '2.0')
(1550495, '2.0')
(2659468, '1.0')
(2727541, '2.0')
(1894023, '1.0')
(2375720, '1.0')
(2478223, '2.0')
(26580, '1.0')
(2271715, '2.0')
(2121072, '2.0')
(1678410, '1.0')
(3062858, '1.0')
(2605106, '2.0')
(1172211, '1.0')
(1992600, '2.0')
(2028767, '1.0')
(2478209, '2.0')
(2241370, '2.0')
(2789758, '1.0')
(2284135, '1.0')
(2478235, '2.0')
(2320105, '2.0')
(2049542, '1.0')
(1550536, '2.0')
(1992605, '1.0')
(2727563, '1.0')
(1844933, '2.0')
(27191, '1.0')
(2228184, '1.0')
(2028761, '2.0')
(27103, '2.0')
(2271711, '2.0')
(2121068, '2.0')
(2415968, '1.0')
(2659422, '1.0')
(1786434, '2.0')
(2028771, '1.0')
(1733546, '2.0')
(1992599, '2.0')
(2090780, '1.0')
(2228229, '1.0')
(2850821, '1.0')
(1992594, '1.0')
(1294279, '2.0')
(1786427, '1.0')
(27099, '2.0')
(2271707, '2.0')
(2413865, '2.0')
(3062823, '1.0')
(2121076, '1.0')
(2659426, '1.0')
(19925

(1733579, '2.0')
(1733592, '1.0')
(2336985, '2.0')
(2694337, '1.0')
(1948850, '1.0')
(1914183, '1.0')
(2165652, '2.0')
(2320104, '1.0')
(2757959, '1.0')
(2375726, '1.0')
(2605102, '1.0')
(2028793, '2.0')
(2757881, '2.0')
(2228223, '2.0')
(1733537, '1.0')
(2241441, '1.0')
(1750846, '1.0')
(2241479, '2.0')
(2028757, '1.0')
(1893986, '2.0')
(3000303, '2.0')
(2789748, '1.0')
(1844913, '1.0')
(2004678, '2.0')
(2320142, '1.0')
(2757963, '1.0')
(2432764, '1.0')
(1616377, '1.0')
(1992580, '1.0')
(2878200, '2.0')
(2028789, '2.0')
(3000315, '2.0')
(2121105, '1.0')
(1893936, '1.0')
(26583, '1.0')
(2134625, '1.0')
(2626145, '1.0')
(2241363, '2.0')
(27134, '1.0')
(1611547, '2.0')
(1550498, '2.0')
(2822984, '2.0')
(1857501, '2.0')
(2554225, '1.0')
(2850822, '2.0')
(2955633, '1.0')
(3000311, '2.0')
(2757904, '2.0')
(2165674, '1.0')
(1690712, '1.0')
(1961567, '2.0')
(1786381, '2.0')
(2375743, '1.0')
(2605119, '1.0')
(2157790, '1.0')
(1294299, '1.0')
(467038, '2.0')
(2271687, '1.0')
(2078056, '2.0')
(2

(1844919, '1.0')
(467032, '1.0')
(2990687, '2.0')
(26680, '2.0')
(1570317, '2.0')
(715635, '2.0')
(27104, '1.0')
(2271712, '1.0')
(2573375, '1.0')
(2571776, '2.0')
(2809344, '2.0')
(2659422, '2.0')
(2375739, '2.0')
(2605115, '2.0')
(1961574, '1.0')
(2165660, '1.0')
(2121111, '1.0')
(2659461, '1.0')
(2713907, '1.0')
(2605089, '1.0')
(3000338, '1.0')
(27049, '1.0')
(1842005, '2.0')
(2241370, '1.0')
(2789758, '2.0')
(1733545, '2.0')
(1733584, '1.0')
(2375735, '2.0')
(2605111, '2.0')
(27551, '2.0')
(2320127, '1.0')
(2121100, '1.0')
(2390651, '2.0')
(2375733, '1.0')
(1786370, '2.0')
(2375718, '1.0')
(2605109, '1.0')
(1294274, '1.0')
(2028771, '2.0')
(2554257, '2.0')
(2133354, '2.0')
(2028735, '2.0')
(2028749, '1.0')
(1894014, '1.0')
(2789740, '1.0')
(2121109, '2.0')
(2413850, '1.0')
(2320149, '1.0')
(2045328, '1.0')
(2090812, '2.0')
(2228182, '1.0')
(2004676, '1.0')
(2789750, '2.0')
(2228211, '2.0')
(1678444, '2.0')
(1733549, '1.0')
(1974880, '1.0')
(2415966, '1.0')
(2478208, '2.0')
(259388

(1733570, '1.0')
(2121105, '2.0')
(1992613, '2.0')
(1162904, '2.0')
(2390637, '2.0')
(2078038, '2.0')
(2413910, '2.0')
(2727576, '2.0')
(1550541, '1.0')
(2284158, '1.0')
(2375688, '1.0')
(2850824, '1.0')
(663202, '1.0')
(2727586, '1.0')
(1844913, '2.0')
(1418128, '2.0')
(2271685, '2.0')
(1733590, '1.0')
(858029, '1.0')
(1992609, '2.0')
(663207, '2.0')
(2432757, '1.0')
(2078018, '2.0')
(2271715, '1.0')
(2228200, '2.0')
(1550545, '1.0')
(2320102, '1.0')
(2078044, '2.0')
(1550545, '2.0')
(2413916, '2.0')
(1064988, '1.0')
(2375708, '1.0')
(1294264, '1.0')
(2432767, '2.0')
(2028760, '2.0')
(1733535, '1.0')
(1418124, '2.0')
(2747411, '2.0')
(2659482, '2.0')
(2028739, '1.0')
(858013, '2.0')
(2361698, '1.0')
(1842013, '1.0')
(1772558, '1.0')
(2432746, '1.0')
(2121106, '2.0')
(2466963, '1.0')
(2375731, '2.0')
(2605107, '2.0')
(1961581, '1.0')
(858048, '2.0')
(27555, '1.0')
(2165667, '1.0')
(2413864, '2.0')
(2415956, '1.0')
(2194922, '2.0')
(2727544, '1.0')
(2241377, '1.0')
(26582, '2.0')
(26261

(2727550, '1.0')
(1992603, '2.0')
(1948829, '1.0')
(2727562, '2.0')
(2727543, '2.0')
(1992592, '2.0')
(486853, '2.0')
(2478137, '1.0')
(1786276, '2.0')
(3000307, '2.0')
(27553, '2.0')
(2078046, '1.0')
(2165665, '2.0')
(2413918, '1.0')
(27145, '2.0')
(2727572, '1.0')
(1733560, '1.0')
(2228217, '2.0')
(1844909, '2.0')
(858015, '1.0')
(1064979, '2.0')
(1961613, '1.0')
(2004679, '2.0')
(1786272, '2.0')
(2413863, '1.0')
(1844936, '1.0')
(2415970, '2.0')
(886979, '1.0')
(2375694, '1.0')
(27141, '2.0')
(2878201, '1.0')
(2028811, '2.0')
(1857508, '2.0')
(2757899, '2.0')
(1786262, '1.0')
(3134186, '1.0')
(2932449, '2.0')
(1948838, '1.0')
(500751, '2.0')
(2727554, '2.0')
(2413867, '1.0')
(2432732, '1.0')
(27097, '1.0')
(2271705, '1.0')
(2932444, '1.0')
(3239286, '1.0')
(1678451, '1.0')
(2320152, '2.0')
(1844940, '2.0')
(27137, '2.0')
(2878206, '1.0')
(3134208, '1.0')
(1435116, '1.0')
(1857511, '1.0')
(1411083, '1.0')
(26635, '1.0')
(26640, '2.0')
(3000336, '2.0')
(663205, '1.0')
(26578, '2.0')
(

In [15]:
df.shape

(598805, 57)

In [16]:
255680/622181

0.41094151058936224

In [17]:
DATA_DIR

'C:\\Users\\Weiguan\\Dropbox\\Research\\DeepHedging\\Data\\Euroxx\\'

In [18]:
clean_dir = DATA_DIR + 'CleanData/'

In [19]:
df.to_csv(clean_dir + f'Options_with_Liquidity_{FREQ}.csv')